In [14]:
import duckdb

search_term = "logic-programming"

# Step 1: Load JSON into a DuckDB temp table with parallel processing enabled
con = duckdb.connect(database=':memory:')
con.execute("SET threads TO 10;")  # Adjust number based on your CPU cores
con.execute("""
    CREATE TEMP TABLE repo AS 
    SELECT * FROM read_json_auto('../public/data/repo_metadata.json');
""")

# Step 2: Get nameWithOwner and topics into a pandas DataFrame
query = "SELECT nameWithOwner, topics FROM repo"
df = con.execute(query).fetchdf()

# Step 3: Normalize topics into list of names
def extract_names(item_ls):
    if item_ls is not None and len(item_ls) > 0:
        return [item["name"] for item in item_ls if "name" in item]
    return []

df["topics"] = df["topics"].apply(extract_names)

# Step 4: Filter repos based on search term in topics
filtered_df = df[df["topics"].apply(lambda x: search_term in [t.lower() for t in x])]

# Step 5: Count all co-occurring topics
all_topics = [topic for topics in filtered_df["topics"] for topic in topics]
topic_counts = Counter(all_topics)

# Remove the searched topic itself
topic_counts.pop(search_term, None)

# Step 6: Convert to list of dicts and sort, only including topics with count > 1
topics = [{"name": name, "count": count} for name, count in topic_counts.items() if count > 2]
topics = sorted(topics, key=lambda x: x["count"], reverse=True)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))